In [2]:
from utils import *

hete = Heterogeneity()
hete.check_torch_gpu()
#fluvial_dataset, gaussian_dataset = hete.load_datasets()

-------------------------------------------------
------------------ VERSION INFO -----------------
Conda Environment: torchy | Python version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
Torch version: 2.0.1
Torch build with CUDA? True
# Device(s) available: 1, Name(s): Quadro P520



In [3]:
my_dir       = 'C:/Users/381792/Documents/H2-Proxy/heterogeneity/'
fluvial_dir  = '//dcstorage.lanl.gov/mfr/misael/depleted_gas_reservoir/2D_Fluvial'
gaussian_dir = '//dcstorage.lanl.gov/mfr/misael/depleted_gas_reservoir/2D_Gaussian/'

In [6]:
Nreal, times, nx, ny, nz = 1000, 61, 256, 256, 1
dirname, filename = {}, {}
df, dv = {}, {}

folder_path = fluvial_dir

data_arr = np.zeros((1000, 61, 256, 256, 1))

for folder_num in range(1, 1001):
    folder_name = '{}UHSS'.format(folder_num)
    folder_dir = os.path.join(folder_path, folder_name)

    for file_num in range(61):
        file_name = '{}UHSS_{}.FUNRST'.format(folder_num, file_num)
        file_path = os.path.join(folder_dir, file_name)

        with open(file_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                if f"{nx*ny*nz} 'REAL'" in line:
                    key = line[2:10].strip()
                    data = []
                    for _ in range(math.ceil(nx*ny*nz/4)):
                        data.extend(line.strip().split())
                    array = np.array(data, dtype=float).reshape((nx,ny,nz), order='F')
            data_arr[folder_num-1, file_num, :, :, 0] = array

ValueError: could not convert string to float: "'PRESSURE'"

In [5]:
key

'PERMX'

***
# END